This python notebook iterates over comparison rasters to pickle up a range of statistics for future presentation. It depends on compare_hydraulic_model.ipynb to produce the actual model outputs and results.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
output_folder = os.environ['OUTPUT_FOLDER']
output_folder


In [ ]:
from hydrological_connectivity.definitions.definitions_generator_factory import DefinitionsGeneratorFactory
import logging
logging.getLogger().setLevel('WARNING')

definition = DefinitionsGeneratorFactory.get_generator()


In [ ]:
from hydrological_connectivity.definitions.definitions_generator_factory import DefinitionsGeneratorFactory
from hydrological_connectivity.processing.model_result_aggregator import ModelResultAggregator
import logging
from hydrological_connectivity.definitions.model_definitions import DepthModelType, ModelDefinitions
from hydrological_connectivity.postprocessing.model_benchmark_cards import ModelBenchmarkCards
logging.getLogger().setLevel('WARNING')


model_types = ModelDefinitions.get_model_definitions()
# display(model_types)

exclude_1956 = ["1956"]
exclude_non_1956 = ['LBS - Culgoa FP South',
                  'LBS - Culgoa FP North',
                  'LBS - Narran River',
                  'LBS - St George',
                  'Namoi - DS Mollee',
                  'Namoi - Boolcarroll',
                  'Namoi - DS Gunidgera',
                  'Namoi - Merah North', '2013','2016']

# exclude_list = exclude_non_1956
exclude_list = exclude_1956
definition = DefinitionsGeneratorFactory.get_generator(exclude_list)
definition.generate()

product_results={}
overall_results={}
overall_metrics={}

model_types = [
    model for model in model_types if model.depth_model_type != DepthModelType.Simple]

for model_type in model_types:
    model_result_aggregator = ModelResultAggregator(definition, model_type)
    model_benchmark_cards = ModelBenchmarkCards(model_result_aggregator, output_folder)
    if model_benchmark_cards.output_exists():
        logging.warning(f"Overwriting model (already exists) {model_type}")
    display(f"Running {model_type}")
    try:
        model_benchmark_cards.run_model()
        model_benchmark_cards.save_histogram()
        model_benchmark_cards.save_hex_maps()
        model_benchmark_cards.save_objects()
        product_results[model_type] = model_benchmark_cards.scene_stats
        if len(model_benchmark_cards.overall_scene_stats)>0:
            overall_results[model_type] = model_benchmark_cards.overall_scene_stats
        if len(model_benchmark_cards.scene_metrics)>0:
            overall_metrics[model_type] = model_benchmark_cards.scene_metrics
        model_benchmark_cards.save_maps()
        if len(model_benchmark_cards.tasks)>0:
            if (exclude_list == exclude_non_1956):
                model_benchmark_cards.save_closeup_map(0)
            else:
                if (model_type.depth_model_type == DepthModelType.FwDET) or (model_type.depth_model_type == DepthModelType.TVD):
                    for i in range(0, len(model_benchmark_cards.tasks)):
                        model_benchmark_cards.save_closeup_map(i)
                else:
                    model_benchmark_cards.save_closeup_map(0)
    except:
        logging.exception(f"Raised an error with {model_type}")


Create a HAND definition for best

In [ ]:
from hydrological_connectivity.definitions.model_definitions import DepthModelType, ModelDefinition
import pandas 
hand_outputs = [x for x in overall_results.keys() if x.depth_model_type==DepthModelType.HAND]

#overall_results[hand_outputs[0]]
#product_results[hand_outputs[0]]
product_results_aggregate = []
for k in hand_outputs:
    v = product_results[k]
    params_str = ", ".join([f"{val}" for (
        att, val) in k.depth_model_params.items()])
    if len(params_str) > 0:
        params_str = f" ({params_str})"

    for (k2, v2) in v.items():
        simple_str = f"{k.depth_model_type.name}{params_str}"
        v2['simulation'] = k2
        v2['model'] = simple_str
        product_results_aggregate.append(v2)

product_results_aggregate_df = pandas.DataFrame(product_results_aggregate)[
    ['model', 'simulation', 'mean', 'std', 'b','perc']]

product_results_aggregate_df['abs-mean'] = product_results_aggregate_df['mean'].abs()

hand_best_by_std_df = product_results_aggregate_df.loc[product_results_aggregate_df.groupby(['simulation'])['std'].idxmin()]
hand_best_by_mean_df = product_results_aggregate_df.loc[product_results_aggregate_df.groupby(
    ['simulation'])['abs-mean'].idxmin()]

hand_best_by_mean_percentiles = pandas.DataFrame(hand_best_by_mean_df.sort_index().perc.tolist(), columns=[
    '2.5', '15.9', '25', '50', '75', '84.1', '97.5'], index=hand_best_by_mean_df.index)
hand_best_by_mean_exploded = hand_best_by_mean_df.join(
    hand_best_by_mean_percentiles)
del hand_best_by_mean_exploded['perc']
#display(product_results_aggregate_df)
#display(hand_best_by_mean_exploded)
import re
accumulation_threshold_lookup = {x: re.sub('\)','',re.sub("HAND \(",'', hand_best_by_mean_exploded[hand_best_by_mean_exploded['simulation'] == x.short_description]['model'].values[0]))  for x in definition.hand_outputs}

model_type = ModelDefinition(
    DepthModelType.HAND, {"accumulation_threshold": {'best MAE': accumulation_threshold_lookup}})
model_result_aggregator = ModelResultAggregator(definition, model_type)
model_benchmark_cards = ModelBenchmarkCards(
    model_result_aggregator, output_folder)
if model_benchmark_cards.output_exists():
    logging.warning(f"Skipping model (already exists) {model_type}")

display(f"Running {model_type}")
model_benchmark_cards.run_model()
model_benchmark_cards.save_histogram()
model_benchmark_cards.save_hex_maps()
model_benchmark_cards.save_objects()
product_results[model_type] = model_benchmark_cards.scene_stats
if len(model_benchmark_cards.overall_scene_stats) > 0:
    overall_results[model_type] = model_benchmark_cards.overall_scene_stats
if len(model_benchmark_cards.scene_metrics) > 0:
    overall_metrics[model_type] = model_benchmark_cards.scene_metrics
model_benchmark_cards.save_maps()
if (exclude_list == exclude_non_1956):
    model_benchmark_cards.save_closeup_map(0)
else:
    for i in range(0, len(model_benchmark_cards.tasks)):
        model_benchmark_cards.save_closeup_map(i)


In [ ]:
import pickle

output = open(
    f'{output_folder}{os.path.sep}product_results.pkl', 'wb')
pickle.dump(product_results, output)
output.close()
output = open(
    f'{output_folder}{os.path.sep}overall_results.pkl', 'wb')
pickle.dump(overall_results, output)
output.close()
output = open(
    f'{output_folder}{os.path.sep}overall_metrics.pkl', 'wb')
pickle.dump(overall_metrics, output)
output.close()
